In [29]:
import tweepy
import pandas as pd
import numpy as np

In [30]:
consumer_key="YpWcXe4P2MaMsffVdeuwwr3nM"
consumer_secret="yG5yFfCWprcBvgUv5GTTmdw8ROOHJrRiRR3ca2FgruqKsgFJHP"
access_token="1276589112615596032-kEXUvreUqCy2osZsMVeEiWqWrREiIP"
access_token_secret="1szE25rTB4Sxfm9DcLumvci1Haoa7MUao8xjA9j8xws1s"
my_bearer_token ="AAAAAAAAAAAAAAAAAAAAAATSXAEAAAAAiBNyVM8QkZRN8mvN605Om40Ao%2B0%3D1mYW2Kws0PGXYIfwfyqwAW5mkm48wl9fc0bX1ucuUPI3AbkXc6"

In [31]:
# create your client 
client = tweepy.Client(bearer_token=my_bearer_token)

In [80]:
qw=input()

#tokyo


In [81]:
# query to search for tweets
query = qw+" lang:en"
query

'#tokyo lang:en'

In [82]:
# your start and end time for fetching tweets
#start_time = "2021-12-10T00:00:00Z"
#end_time = "2021-12-14T00:00:00Z"

# get tweets from the API
tweets = client.search_recent_tweets(query=query,
                                     #start_time=start_time,
                                     #end_time=end_time,
                                     tweet_fields = ["created_at", "text","public_metrics","entities"],
                                     user_fields = ["name", "username", "location", "verified", "description","public_metrics"],
                                     max_results = 100,
                                     expansions='author_id'
                                     )

In [83]:
# tweet specific info
print(len(tweets.data))
# user specific info
print(len(tweets.includes["users"]))

100
50


In [84]:
first_tweet = tweets.data[10]
dict(first_tweet)

{'entities': {'hashtags': [{'start': 73, 'end': 79, 'tag': 'Tokyo'}],
  'urls': [{'start': 80,
    'end': 103,
    'url': 'https://t.co/QpxERwpmNo',
    'expanded_url': 'https://twitter.com/tokyo_todd/status/1472873363622498304/photo/1',
    'display_url': 'pic.twitter.com/QpxERwpmNo'}]},
 'text': 'Don’t worry, the skytube probably comes in peace. Just do what it says. \n#Tokyo https://t.co/QpxERwpmNo',
 'created_at': datetime.datetime(2021, 12, 20, 10, 15, 49, tzinfo=datetime.timezone.utc),
 'public_metrics': {'retweet_count': 0,
  'reply_count': 0,
  'like_count': 2,
  'quote_count': 0},
 'id': 1472873363622498304,
 'author_id': 192101447}

In [85]:
# user information for the first tweet
first_tweet_user = tweets.includes["users"][10]
dict(first_tweet_user)

{'location': 'Hidaka-shi, Saitama',
 'username': 'tokyo_todd',
 'verified': False,
 'id': 192101447,
 'public_metrics': {'followers_count': 621,
  'following_count': 1141,
  'tweet_count': 4429,
  'listed_count': 22},
 'description': 'Writer, editor, strummer of guitars, collector of books and rocks, hugger of trees, taker of less-traveled roads. Living in Japan since 1990.',
 'name': 'Todd Fouts'}

In [86]:
#dict(first_tweet)
tg=[]
for i in first_tweet['entities']['hashtags']:
        tg.append(i['tag'])
print(str(tg))
type(tg)

['Tokyo']


list

In [87]:
def tags(t):
    tgs=[]
    ett=dict(t)
    if 'hashtags' in ett['entities']:
        for i in ett['entities']['hashtags']:
            tgs.append(i['tag'])
    return tgs

In [88]:
# create a list of records
tweet_info_ls = []
# iterate over each tweet and corresponding user details
for tweet, user in zip(tweets.data, tweets.includes['users']):
    tweet_info = {
        'created_at': tweet.created_at,
        'text': tweet.text,
        'retweet_count': tweet.public_metrics['retweet_count'],
        'like_count' : tweet.public_metrics['like_count'],
        'hashtags' : tags(tweet),
        'name': user.name,
        'username': user.username,
        'location': user.location,
        'verified': user.verified,
        'description': user.description,
        'following' : user.public_metrics['following_count'],
        'followers' : user.public_metrics['followers_count'],
        'totaltweets' : user.public_metrics['tweet_count']
    }
    tweet_info_ls.append(tweet_info)
# create dataframe from the extracted records
tweets_df = pd.DataFrame(tweet_info_ls)
# display the dataframe
tweets_df.head()

,created_at,text,retweet_count,like_count,hashtags,name,username,location,verified,description,following,followers,totaltweets
0,2021-12-20 10:36:11+00:00,RT @yonan_ichan: Juzu glow art \n\n📌Open for A...,3,0,"[anime, AnimeArt, animetwt, animefanart]",tan’s art acc 🌤 ia for a bit,arts_by_tan,None,False,"ig: arts_by_tan , he/him/they, 🇵🇭, haii I am T...",1033,1438,7840
1,2021-12-20 10:34:40+00:00,RT @FrauHodl: 🟣 .com-Domain for sale\n\nhttps:...,3,0,"[Metaverse, virtual, AI, MachineLearning, bran...",FrauHodl.x 💃🏻,FrauHodl_CMO,None,False,Web3NFT | Web3 | Advisory | xxxx.888 #Web3 #Un...,1022,267,185
2,2021-12-20 10:30:15+00:00,RT @popuptravelexp: https://t.co/xf94BmtcpW BO...,1,0,"[adventure, travel, flight]",Popup Travel Experiences,popup_travel,"Sydney, New South Wales",False,Pop up travel experience - tours - hotels http...,173,94,659
3,2021-12-20 10:29:57+00:00,#NowPlaying DJ Tag - DJ Layne on @BN4IA #Radio...,0,0,"[NowPlaying, Radio, Tokyo]",BN4IA,BN4IA,Las Vegas,False,Broadcast Network for Indie Artists - Via FTA ...,1654,1224,72107
4,2021-12-20 10:29:34+00:00,RT @yonan_ichan: Juzu glow art \n\n📌Open for A...,3,0,"[anime, AnimeArt, animetwt, animefanart]",📌❄ r o n i. comms open. 🖤,roniiiqdr,ig:@roniiiqdr|| She/Her || 21+,False,I draw—. || PROMO ongoing! || coffee?,1118,1196,7042


In [89]:
tweets_df.to_csv(r'C:\Users\Pranjal Gupta\Desktop\Twitter data\output.csv', index=False)